In [1]:
from keras.layers import Input, Lambda, Dense, Flatten,GlobalAveragePooling2D
from keras.models import Model
from keras.applications import MobileNet
from keras.applications.mobilenet import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dropout
from keras.models import Sequential
import numpy as np
from glob import glob

In [2]:
# Mobilenet accepts input size (224,224) 
# remove the last layer 
mb = MobileNet(input_shape=(224,224,3), weights='imagenet', include_top=False)

In [3]:
# to avoid training the already trained layers of Mobilenet
for layer in mb.layers:
  layer.trainable = False

In [4]:
# checking the folders of images
folders = glob('drive/My Drive/train/*')
folders

['drive/My Drive/train/cats', 'drive/My Drive/train/dog']

In [5]:
x = mb.output
# adding more layers to be trained on
x=GlobalAveragePooling2D()(x) 
x = Dense(1024, activation ='relu')(x)
x=Dense(512,activation='relu')(x)
prediction=Dense(2, activation='softmax')(x)

In [6]:
model = Model(inputs=mb.input, outputs=prediction)
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)     

In [7]:
# compile the model
model.compile(
  loss='categorical_crossentropy',
  optimizer='sgd',
  metrics=['accuracy']
)

In [8]:
# train image augmentation
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
training_set = train_datagen.flow_from_directory('drive/My Drive/train',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 600 images belonging to 2 classes.


In [9]:
# test image augmentation
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
test_set = test_datagen.flow_from_directory('drive/My Drive/test',
                                            target_size = (224,224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 156 images belonging to 2 classes.


In [10]:
# classes
training_set.class_indices

{'cats': 0, 'dog': 1}

In [11]:
model.fit_generator(training_set, steps_per_epoch=600/32, validation_data = test_set, validation_steps=156, epochs = 5)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/5
19/18 [==============================] - 4s 231ms/step - loss: 0.3369 - accuracy: 0.8600 - val_loss: 0.1641 - val_accuracy: 0.9551
Epoch 2/5
19/18 [==============================] - 3s 149ms/step - loss: 0.1157 - accuracy: 0.9750
Epoch 3/5
19/18 [==============================] - 3s 148ms/step - loss: 0.0783 - accuracy: 0.9867
Epoch 4/5
19/18 [==============================] - 3s 148ms/step - loss: 0.0581 - accuracy: 0.9883
Epoch 5/5
19/18 [==============================] - 3s 150ms/step - loss: 0.0465 - accuracy: 0.9917


In [12]:
# testing the image of a cat
from keras.preprocessing import image
import numpy as np
test_img=image.load_img('drive/My Drive/dataset/single_prediction/cat_or_dog_2.jpg', target_size=(224,224))
test_img=image.img_to_array(test_img)
test_img=np.expand_dims(test_img, axis=0)
result=model.predict(test_img/255, batch_size=10)
print("The image belongs to class ", np.argmax(result))

The image belongs to class  0


In [13]:
result = model.evaluate(test_set)
print('The model achieved a loss of %.2f and,'
      'accuracy of %.2f%%.' % (result[0], result[1]*100)) 

5/5 [==============================] - 1s 123ms/step - loss: 0.1023 - accuracy: 0.9615
The model achieved a loss of 0.10 and,accuracy of 96.15%.
